In [2]:
from google.colab import files
uploaded = files.upload()

Saving Telco_cleaned.csv to Telco_cleaned.csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('Telco_cleaned.csv')

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Create and train the baseline SVC
svc = SVC(class_weight='balanced')  # Automatically adjusts weights
svc.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = svc.predict(X_test_scaled)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[730 303]
 [ 88 286]]
              precision    recall  f1-score   support

           0       0.89      0.71      0.79      1033
           1       0.49      0.76      0.59       374

    accuracy                           0.72      1407
   macro avg       0.69      0.74      0.69      1407
weighted avg       0.78      0.72      0.74      1407



In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [12],
    'gamma': [1,0.9,0.8],
    'kernel': ['linear']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, cv=5, scoring='f1')
grid.fit(X_train_scaled, y_train)

# Best parameters
print("Best Parameters:", grid.best_params_)

# Evaluate best model
y_pred_grid = grid.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))


Best Parameters: {'C': 12, 'gamma': 1, 'kernel': 'linear'}
[[917 116]
 [168 206]]
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1033
           1       0.64      0.55      0.59       374

    accuracy                           0.80      1407
   macro avg       0.74      0.72      0.73      1407
weighted avg       0.79      0.80      0.79      1407



In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

# Rebuild pipeline with your best parameters
best_params = grid.best_params_

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(**best_params))
])

# Fit the pipeline on the original training data
pipeline.fit(X_train, y_train)

# Save pipeline
joblib.dump(pipeline, 'churn_pipeline.pkl')


['churn_pipeline.pkl']

In [10]:
files.download('churn_pipeline.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>